In [8]:
import numpy as np

In [1]:
from gensim.models import KeyedVectors

C:\Users\lappy\Anaconda21\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
vecfile = 'GoogleNews-vectors-negative300.bin'
vecs = KeyedVectors.load_word2vec_format(vecfile, binary=True)


C:\Users\lappy\Anaconda21\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
def compute_cosine_similarity(vector1, vector2):
    
    cos = vector1.dot(vector2) / (np.linalg.norm(vector1, ord=2) * np.linalg.norm(vector2, ord=2))
    if np.isnan(cos):
        return 0.500    # arbitrarily low semantic similarity
    else:
        return cos

In [4]:
def w2v_semantic_sim(input, vecs, output):
    write_str = []
    sims = []
    s_min = 1
    s_max = 0

    for line in input:
        line_components = line.split("\t")
        sent1 = line_components[0]
        sent2 = line_components[1]
        words_in_sent1 = sent1.split()
        words_in_sent2 = sent2.split()
        v1 = np.zeros(vecs["hi"].shape)
        for word in words_in_sent1:
            if word in vecs:
                v1 = v1 + np.asarray(vecs[word])

        v2 = np.zeros(vecs["hi"].shape)
        for word in words_in_sent2:
            if word in vecs:
                v2 = v2 + np.asarray(vecs[word])

        sim = compute_cosine_similarity(v1, v2)

        write_str.append(sent1 + "\t" + sent2 + "\t")
        sims.append(sim)

        s_max = max(s_max, sim)
        s_min = min(s_min, sim)

    sims_scaled = [5*(i - s_min)/(s_max - s_min) for i in sims]
    for i in range(0,len(write_str)):
        output.write(write_str[i] + str(sims_scaled[i]) + "\n")


In [9]:
with open('en-test.txt', 'r') as inputfile:
        input = inputfile.readlines()
        
output_simple = open("pred_simple.txt", "w")

w2v_semantic_sim(input, vecs, output_simple)
output_simple.close()